In [ ]:
# default_exp funcs.merfish_comparison

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from decode_fish.imports import *

# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.plotting import *
from decode_fish.engine.noise import estimate_noise_scale
import shutil
from decode_fish.funcs.visualization import *
from decode_fish.funcs.predict import predict
import torch.nn as nn
import torch.nn.functional as F
import torch.tensor as T
from decode_fish.funcs.predict import *

from omegaconf import open_dict
from hydra import compose, initialize
from decode_fish.funcs.merfish_eval import *

sys.path.append('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/FQ/istdeco/')

from istdeco import ISTDeco
from utils import random_codebook, random_image_stack
from codebook import Codebook
from starfish.image import Filter

sys.path.append('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/FQ/bardensr/')
import bardensr

import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

2021-10-05 03:45:40.574215: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_ci_13/start_mic:5000xpsf_noise:0.4/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sl_9a/old_new_format/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sl_11/seed:1xint_fac:0.1/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_1/old_loss:Truexmin_int_sig:1.0/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_1/old_loss:False/train.yaml')
# csv_name = 'test'
# cfg.foci.n_foci_avg = 0

In [ ]:
# cfg.exp_type.pred_z=False
# cfg.PSF.psf_extent_zyx=[1,21,21]
# cfg.run_name = 'test'

In [ ]:
from decode_fish.funcs.merfish_eval import *
bench_df, code_ref, targets = get_benchmark()
code_inds = np.stack([np.nonzero(c)[0] for c in code_ref])

13832


In [ ]:
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_2/test1/train.yaml')
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sim_1b/old_loss:Falsexpsf_noise:0.0xint_fac:0.0/train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_cinfsim_1/depth:2xf_maps:256///train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_cinfsim_1c/loss_option:1xcount_mult:0xcat_logits:0//train.yaml')
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sl_16b/psf_noise:0.15xenabled:False///train.yaml')
# cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/sweep_sl_14c/loss_option:0xcount_mult:1xcat_logits:1/train.yaml')

In [ ]:
# cfg = OmegaConf.load('../config/experiment/MERFISH_ci_2d_sim1.yaml')
# cfg.output.save_dir = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_starfish/nb_run/test/'

In [ ]:
crop_sz = 128
cfg.foci.n_foci_avg = 0
cfg.training.bs = 5
cfg.PSF.n_cols = 1
cfg.random_crop.crop_sz = crop_sz
cfg.data_path.image_shape = [16,1,crop_sz,crop_sz]
cfg.model.n_p_layers = 1

model, post_proc, micro, img_3d, decode_dl = load_all(cfg)
psf, noise, micro = load_psf_noise_micro(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 128, 128)
1 volumes


In [ ]:
from decode_fish.engine.point_process import *
from decode_fish.funcs.output_trafo import *
from decode_fish.funcs.evaluation import *
import h5py
from decode_fish.funcs.routines import *

with torch.no_grad():

    x, local_rate, background = next(iter(decode_dl))
    sim_vars = PointProcessUniform(local_rate*0 + 0.02, model.int_dist.int_conc, model.int_dist.int_rate, 
                                   model.int_dist.int_loc, channels=16, n_bits=4, sim_z=True, codebook=torch.tensor(code_inds), int_option=3).sample(from_code_book=True, phasing=cfg.exp_type.phasing)
    xsim = micro(*sim_vars[:-1], add_noise=True)
    xsimn = micro.noise(xsim, background*0 + 100).sample()

    gt_vars = sim_vars[:-1]
    gt_df = px_to_nm(sample_to_df(*sim_vars[:-2], codes=sim_vars[-1], px_size_zyx=[1.,1.,1.]))

# with h5py.File('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_data1/dens_0.1_int_1.0.h5', 'r') as f:

#     xsimn = f['frames'][()]
#     gt_df = get_df_from_hdf5(f['locations']) 
        
# with h5py.File('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_data1/sim_2/dens_0.1_int_1.0.h5', 'r') as f:

#     dec_df = get_df_from_hdf5(f['old_loss:False']['locations_int'])

In [ ]:
# gt_df = get_code_from_ints(gt_df, code_ref, targets, int_str='', p_str='')
# gt_df = exclude_borders(gt_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

In [ ]:
# gt_df = gt_df.loc[:,['frame_idx', 'code_inds', 'x', 'y', 'z']+[f'int_{i}' for i in range(16)]]
# np.save('../data/chen_bardensr/sim_data.npy',cpu(xsimn))
# gt_df.to_csv('../data/chen_bardensr/gt_df.csv', index=False)
# np.save('../data/chen_bardensr/codebook.npy',cpu(code_ref))

In [ ]:
model, post_proc, micro, img_3d, decode_dl = load_all(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 128, 128)
1 volumes


In [ ]:
with torch.no_grad():
    
    model.eval().cuda()
    res_dict = model(T(xsimn).cuda())
    res_dict = model.tensor_to_dict(res_dict)
    pred_df = post_proc.get_df(res_dict, softmax=False)
#     pred_df = get_code_from_ints(pred_df, code_ref, targets, func=vcorrcoef, int_str='', p_str='')
#     pred_df = exclude_borders(pred_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
# for c in range(30):
# #     c = 72

#     sub_gt = gt_df[gt_df['frame_idx'] == 1]
#     sub_gt = sub_gt[sub_gt['code_inds'] == c]
    
#     sub_pred = pred_df[pred_df['frame_idx'] == 1]
#     sub_pred = sub_pred[sub_pred['code_inds'] == c]

#     plt.figure(figsize=(10,10))
#     im = plt.imshow(cpu(torch.sigmoid(res_dict['logits']))[1,c,0])
#     add_colorbar(im)
#     plt.scatter(sub_gt['x']/100, sub_gt['y']/100, ec='red', s=100, marker='+')
#     plt.scatter(sub_pred['x']/100, sub_pred['y']/100, ec='green', fc='none', s=100, marker='o', linewidth=3)
#     plt.show()

In [ ]:
sub_df = percentile_filter(pred_df, 100, 'comb_sig')
perf, matches, _  = matching(gt_df, sub_df, match_genes=False, tolerance=300)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True, tolerance=300)

Recall: 0.932
Precision: 0.886
Jaccard: 83.239
RMSE_vol: 66.413
Eff_3d: 62.803
FN: 108.0 FP: 189.0
Num. matches: 1475.000
Shift: 1.27,-0.61,51.79
------- 0.9823728813559321
Recall: 0.918
Precision: 0.873
Jaccard: 80.992
RMSE_vol: 61.783
Eff_3d: 63.729
FN: 130.0 FP: 211.0
Num. matches: 1453.000
Shift: 1.28,-0.42,51.73


In [ ]:
# export
def get_istdeco_df(sim_data, codebook, psf_sig=(1.7, 1.7), n_iter=100, bg=100.):
    
    istd_results = pd.DataFrame()
    
    n_rounds = codebook.shape[1]
    n_cols = codebook.shape[2]
    
    for i in range(len(sim_data)):

        image_data = np.array(sim_data)[i,:,0]
        image_data = image_data.reshape([n_rounds,n_cols,image_data.shape[-2],image_data.shape[-1]], order='F')
        
        init_th = np.percentile(image_data,50)

        istdeco_model = ISTDeco(image_data, codebook, psf_sig, b=bg)
        X, Q, loss = istdeco_model.run(niter=n_iter)

        # Get codes
        code_id, y, x = np.where(np.logical_and(X>init_th,Q>0.1))

        intensity = X[code_id, y, x]
        quality = Q[code_id, y, x]

        # Store in dataframe
        df = pd.DataFrame(data={
                'frame_idx': i,
                'x': x.astype('float')+0.5, 
                'y': y.astype('float')+0.5,
                'z': 0.5,
                'intensity': intensity, 
                'quality': quality,
                'code_inds': code_id
        })

        istd_results = istd_results.append(df)

    istd_results = px_to_nm(istd_results)

    return istd_results

In [ ]:
istd_df = get_istdeco_df(cpu(xsimn), code_ref.reshape([140,8,2], order='F'), psf_sig=(1.7, 1.7), n_iter=400)
# istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

In [ ]:
sub_df = istd_df[istd_df['quality'] > 0.55]
sub_df = sub_df[sub_df['intensity'] > 350]

In [ ]:
perf, matches, _  = matching(gt_df, sub_df, match_genes=False, tolerance=300)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True, tolerance=300)

Recall: 0.980
Precision: 0.969
Jaccard: 94.979
RMSE_vol: 76.533
Eff_3d: 61.405
FN: 32.0 FP: 50.0
Num. matches: 1551.000
Shift: 0.03,1.50,-1.52
------- 0.9800128949065119
Recall: 0.977
Precision: 0.966
Jaccard: 94.502
RMSE_vol: 76.120
Eff_3d: 61.545
FN: 36.0 FP: 54.0
Num. matches: 1547.000
Shift: -0.20,1.45,-1.54


In [ ]:
istd_df = get_istdeco_df(cpu(xsimn), code_ref.reshape([140,8,2], order='F'), psf_sig=(1.7, 1.7), n_iter=400)
# istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

q_max = 0.3*istd_df['quality'].max()
i_max = 0.3*istd_df['intensity'].max()
i_min = istd_df['intensity'].min()

def objective(trial):
    
    n_iter = 400 # trial.suggest_uniform('n_iter', 50, 500)
    psf_s = 1.7 # trial.suggest_uniform('psf_s', 1.5, 1.7)
    
#     istd_df = get_istdeco_df(xsimn, code_ref.reshape([140,8,2], order='F'), psf_sig=(psf_s, psf_s), n_iter=int(n_iter))
#     istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
    
    qual_th = trial.suggest_uniform('qual_th', 0.1, q_max)
    int_th = trial.suggest_uniform('int_th', i_min, i_max)
    
    sub_df = istd_df[(istd_df['intensity'] > int_th) & (istd_df['quality'] > qual_th)]
    perf, matches, _  = matching(gt_df, sub_df, match_genes=True, print_res=False) 
    
    return -perf['jaccard']

study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)
print('Jaccard ', -study.best_value)

[I 2021-10-04 07:42:54,248] A new study created in memory with name: no-name-2b35cf76-7b34-4c14-9f2c-b39ad890b568
[I 2021-10-04 07:42:54,282] Trial 0 finished with value: -0.1092896042149843 and parameters: {'qual_th': 0.13290640735882736, 'int_th': 3774.5292021449723}. Best is trial 0 with value: -0.1092896042149843.
[I 2021-10-04 07:42:54,311] Trial 1 finished with value: -0.03458924835216203 and parameters: {'qual_th': 0.3983130894117003, 'int_th': 5053.285288963072}. Best is trial 0 with value: -0.1092896042149843.
[I 2021-10-04 07:42:54,346] Trial 2 finished with value: -0.04582338464693225 and parameters: {'qual_th': 0.38578318228084474, 'int_th': 1199.5615636800203}. Best is trial 0 with value: -0.1092896042149843.
[I 2021-10-04 07:42:54,377] Trial 3 finished with value: -0.0387215686881907 and parameters: {'qual_th': 0.3999929044591227, 'int_th': 4550.351919498556}. Best is trial 0 with value: -0.1092896042149843.
[I 2021-10-04 07:42:54,407] Trial 4 finished with value: -0.0871

[I 2021-10-04 07:42:55,884] Trial 37 finished with value: -0.11893433692346937 and parameters: {'qual_th': 0.25057006758606637, 'int_th': 1365.4742820855322}. Best is trial 31 with value: -0.25131387947311234.
[I 2021-10-04 07:42:55,923] Trial 38 finished with value: -0.15192742041639223 and parameters: {'qual_th': 0.1474901937334432, 'int_th': 2771.897281703781}. Best is trial 31 with value: -0.25131387947311234.
[I 2021-10-04 07:42:55,958] Trial 39 finished with value: -0.036461442144477926 and parameters: {'qual_th': 0.4270652185047952, 'int_th': 3458.4925218764133}. Best is trial 31 with value: -0.25131387947311234.
[I 2021-10-04 07:42:55,994] Trial 40 finished with value: -0.09442058928012394 and parameters: {'qual_th': 0.11954729010832658, 'int_th': 4159.520511249009}. Best is trial 31 with value: -0.25131387947311234.
[I 2021-10-04 07:42:56,039] Trial 41 finished with value: -0.21821819637455492 and parameters: {'qual_th': 0.14730581974725784, 'int_th': 1696.4676642757863}. Best

[I 2021-10-04 07:42:57,882] Trial 74 finished with value: -0.2743555898335002 and parameters: {'qual_th': 0.10035218290872235, 'int_th': 1205.9429570044824}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:42:57,964] Trial 75 finished with value: -0.21078861267730206 and parameters: {'qual_th': 0.11129156121790101, 'int_th': 604.795501015285}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:42:58,021] Trial 76 finished with value: -0.27368864012786465 and parameters: {'qual_th': 0.10041413798732915, 'int_th': 1155.4838940024529}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:42:58,061] Trial 77 finished with value: -0.0484330438335191 and parameters: {'qual_th': 0.37861760158051977, 'int_th': 1182.404677965084}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:42:58,113] Trial 78 finished with value: -0.24603871073602898 and parameters: {'qual_th': 0.1360147681444494, 'int_th': 1209.0277431236168}. Best is tri

[I 2021-10-04 07:42:59,965] Trial 112 finished with value: -0.26266193461625953 and parameters: {'qual_th': 0.10790378119654274, 'int_th': 1038.4589405708464}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:00,014] Trial 113 finished with value: -0.2630364316533058 and parameters: {'qual_th': 0.10816986886866237, 'int_th': 1360.7184637668547}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:00,063] Trial 114 finished with value: -0.2515137166731517 and parameters: {'qual_th': 0.1288640148931493, 'int_th': 1385.528727622256}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:00,107] Trial 115 finished with value: -0.21499997850000216 and parameters: {'qual_th': 0.153062797946946, 'int_th': 1692.787154743468}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:00,151] Trial 116 finished with value: -0.21432275928149877 and parameters: {'qual_th': 0.1365962814177931, 'int_th': 1895.9121204016212}. Best is 

[I 2021-10-04 07:43:01,970] Trial 149 finished with value: -0.24563104411349088 and parameters: {'qual_th': 0.12713045003808296, 'int_th': 1517.4984542568939}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:02,011] Trial 150 finished with value: -0.02352940955017322 and parameters: {'qual_th': 0.478821448264194, 'int_th': 976.6393021263835}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:02,055] Trial 151 finished with value: -0.06915887204122691 and parameters: {'qual_th': 0.3287687329892431, 'int_th': 1175.328543418909}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:02,112] Trial 152 finished with value: -0.27357291555408747 and parameters: {'qual_th': 0.10009787571881268, 'int_th': 1155.0713069400433}. Best is trial 74 with value: -0.2743555898335002.
[I 2021-10-04 07:43:02,181] Trial 153 finished with value: -0.2538106874292317 and parameters: {'qual_th': 0.10036806229760534, 'int_th': 868.1725920467062}. Best is

[I 2021-10-04 07:43:03,917] Trial 186 finished with value: -0.25855173180348895 and parameters: {'qual_th': 0.12406979257819742, 'int_th': 1235.08504936027}. Best is trial 154 with value: -0.2745270325977094.
[I 2021-10-04 07:43:03,975] Trial 187 finished with value: -0.2591427244087515 and parameters: {'qual_th': 0.11552530757951344, 'int_th': 1037.284253191193}. Best is trial 154 with value: -0.2745270325977094.
[I 2021-10-04 07:43:04,039] Trial 188 finished with value: -0.2536549522182053 and parameters: {'qual_th': 0.10753373668645791, 'int_th': 916.2789857577015}. Best is trial 154 with value: -0.2745270325977094.
[I 2021-10-04 07:43:04,089] Trial 189 finished with value: -0.2597700910556238 and parameters: {'qual_th': 0.119818507513686, 'int_th': 1346.1338242641825}. Best is trial 154 with value: -0.2745270325977094.
[I 2021-10-04 07:43:04,144] Trial 190 finished with value: -0.2672981657909254 and parameters: {'qual_th': 0.1089500287090412, 'int_th': 1115.742430591923}. Best is 

{'qual_th': 0.10038908191941358, 'int_th': 1208.9583165166134}
Jaccard  0.27471563650781833


In [ ]:
sub_df = istd_df[(istd_df['intensity'] > study.best_params['int_th']) & (istd_df['quality'] > study.best_params['qual_th'])]
perf, matches, _  = matching(gt_df, sub_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True)

Recall: 0.595
Precision: 0.729
Jaccard: 48.749
RMSE_vol: 169.596
Eff_3d: 0.918
FN: 649.0 FP: 355.0
Num. matches: 955.000
Shift: 2.48,3.01,0.00
------- 0.5821989528795811
Recall: 0.392
Precision: 0.479
Jaccard: 27.472
RMSE_vol: 50.623
Eff_3d: 23.182
FN: 976.0 FP: 682.0
Num. matches: 628.000
Shift: 1.57,0.01,0.00


In [ ]:
# export
def get_bardensr_tensor(sim_data, codebook, bg=100., n_iter=400, l1_pen=0., psf_r=0):
    
    bard_results = pd.DataFrame()
    evd_tensors = []
    codeflat = codebook.T
    
    for i in range(len(sim_data)):
    
        image_data = sim_data[i]
        Xnorm = image_data - bg
#         Xnorm /= Xnorm.max()
        
#         Xnorm = bardensr.preprocessing.minmax(image_data - bg)

#         Xnorm = bardensr.preprocessing.minmax(image_data)
#         Xnorm = bardensr.preprocessing.background_subtraction(Xnorm,[0,10,10])
        Xnorm = bardensr.preprocessing.minmax(Xnorm)

        evidence_tensor_iterative,extra_learned_params=\
            bardensr.spot_calling.estimate_density_iterative(Xnorm.astype('float64'),codeflat,l1_penalty=l1_pen,use_tqdm_notebook=True,iterations=n_iter, estimate_codebook_gain=True, psf_radius=(psf_r, psf_r, psf_r))

#         evidence_tensor_iterative= bardensr.spot_calling.estimate_density_singleshot(Xnorm.astype('float64'), codeflat, noisefloor=0.05)
    
        evd_tensors.append(evidence_tensor_iterative)
        
    return evd_tensors

def get_bardensr_df(evd_tensors, th, ps=1.0):
    
    bard_results = pd.DataFrame()

    for i in range(len(evd_tensors)):    
    #     thresh_iterative=evd_tensors[i].max()*.1
        result_iterative=bardensr.spot_calling.find_peaks(evd_tensors[i],th, poolsize=(ps,ps,ps))

        code_inds = np.array(result_iterative.j.values, dtype=np.int16)
        df = pd.DataFrame(data={
                    'frame_idx': i,
                    'x': result_iterative.m2.values + 0.5, 
                    'y': result_iterative.m1.values + 0.5,
                    'z': 0.5,
                    'intensity': result_iterative.int.values,
                    'code_inds': code_inds
            })

        bard_results = bard_results.append(df)

    bard_results = px_to_nm(bard_results)
    return bard_results

In [ ]:
evd_tensors = get_bardensr_tensor(cpu(xsimn), code_ref, n_iter=100, l1_pen=.01, bg=100., psf_r=0)

2021-09-25 11:24:06.459223: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-25 11:24:06.460009: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-25 11:24:06.461228: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-09-25 11:24:06.461275: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-25 11:24:06.461340: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-25 11:24:06.461381: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2021-09-25 11

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
bard_df = get_bardensr_df(evd_tensors, th=.25, ps=1.05)
# bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

2021-09-25 11:24:10.373225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [ ]:
len(bard_df)

8159

In [ ]:
perf, matches, _  = matching(gt_df, bard_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, bard_df, match_genes=True)

Recall: 0.858
Precision: 0.850
Jaccard: 74.517
RMSE_vol: 143.815
Eff_3d: 23.711
FN: 1149.0 FP: 1223.0
Num. matches: 6936.000
Shift: -1.75,2.97,0.00
------- 0.5641580161476355
Recall: 0.623
Precision: 0.618
Jaccard: 44.984
RMSE_vol: 81.173
Eff_3d: 31.633
FN: 3045.0 FP: 3119.0
Num. matches: 5040.000
Shift: 0.37,1.18,0.00


In [ ]:
# evd_tensors = get_bardensr_tensor(xsimn, code_ref, n_iter=int(300), l1_pen=0.01)

def objective(trial):
    
#     n_iter = 300
#     l1_pen = trial.suggest_uniform('l1_pen', 0., 0.1)
    th     = trial.suggest_uniform('th', 0.2, 0.6)
    ps     = trial.suggest_uniform('ps', 0.1, 1.5)
    
#     evd_tensors = get_bardensr_tensor(xsimn, code_ref, n_iter=int(n_iter), l1_pen=l1_pen)
#     print(evd_tensors[0].mean())
    bard_df = get_bardensr_df(evd_tensors, th, ps)
    bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
    
    perf, matches, _  = matching(gt_df, bard_df, match_genes=True, print_res=False) 
    
    return -perf['jaccard']

study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)
print('Jaccard ', -study.best_value)

In [ ]:
evd_tensors = get_bardensr_tensor(cpu(xsimn), code_ref, n_iter=100, l1_pen=.05, bg=100., psf_r=0)

In [ ]:
bard_df = get_bardensr_df(evd_tensors, th=study.best_params['th'], ps=study.best_params['ps'])
# bard_df = get_bardensr_df(evd_tensors, th=0.5, ps=1.05)
bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
perf, matches, _  = matching(gt_df, bard_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, bard_df, match_genes=True)

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 20_MERFISH_visualization.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted index.ipynb.
